In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [2]:
import utils

In [3]:
itp_id = 300

In [4]:
pd.set_option("display.max_columns", 100)


In [5]:
#checking to see if rt query works: 

rt = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra-staging.natalie_views.test_rt_trips`
        """
    )
    
rt['str_len'] = rt.trip_id.str.len()

rt.date.value_counts()

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


2022-05-26    1496
2022-05-18    1488
2022-05-04    1487
2022-05-12    1483
2022-05-11    1482
2022-05-10    1482
2022-05-17    1478
2022-05-19    1476
2022-05-03    1473
2022-05-24    1473
2022-05-20    1466
2022-05-05    1455
2022-05-25    1450
2022-05-13    1439
2022-05-27    1417
2022-05-06    1392
2022-05-23    1349
2022-05-31    1323
2022-05-02    1321
2022-05-09    1307
2022-05-16    1285
2022-05-21    1056
2022-05-28    1006
2022-05-07    1003
2022-05-14     982
2022-05-22     835
2022-05-29     818
2022-05-15     813
2022-05-30     800
2022-05-08     787
2022-05-01     779
Name: date, dtype: int64

In [6]:
date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [20]:
# start_date= ['2022-05-01']
# end_date=['2022-05-31']

from datetime import date, timedelta
start_date = date(2022, 5, 1)
end_date = date(2022, 5, 31)

In [ ]:
daterange = pd.date_range(start_date, end_date)

In [29]:
print(daterange)

DatetimeIndex(['2022-05-01', '2022-05-02', '2022-05-03', '2022-05-04',
               '2022-05-05', '2022-05-06', '2022-05-07', '2022-05-08',
               '2022-05-09', '2022-05-10', '2022-05-11', '2022-05-12',
               '2022-05-13', '2022-05-14', '2022-05-15', '2022-05-16',
               '2022-05-17', '2022-05-18', '2022-05-19', '2022-05-20',
               '2022-05-21', '2022-05-22', '2022-05-23', '2022-05-24',
               '2022-05-25', '2022-05-26', '2022-05-27', '2022-05-28',
               '2022-05-29', '2022-05-30', '2022-05-31'],
              dtype='datetime64[ns]', freq='D')


In [34]:
#above range does not work with function

In [43]:
dates = set()

def daterange2(start, end):
    for n in range(int((end - start).days) + 1):
        yield start + timedelta(n)

for single_date in daterange2(start_date, end_date):
    dates.add(single_date.strftime('%Y-%m-%d'))

In [44]:
dates

{'2022-05-01',
 '2022-05-02',
 '2022-05-03',
 '2022-05-04',
 '2022-05-05',
 '2022-05-06',
 '2022-05-07',
 '2022-05-08',
 '2022-05-09',
 '2022-05-10',
 '2022-05-11',
 '2022-05-12',
 '2022-05-13',
 '2022-05-14',
 '2022-05-15',
 '2022-05-16',
 '2022-05-17',
 '2022-05-18',
 '2022-05-19',
 '2022-05-20',
 '2022-05-21',
 '2022-05-22',
 '2022-05-23',
 '2022-05-24',
 '2022-05-25',
 '2022-05-26',
 '2022-05-27',
 '2022-05-28',
 '2022-05-29',
 '2022-05-30',
 '2022-05-31'}

In [45]:
final_range = utils.get_pct_ran_df(dates, itp_id)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/l

In [46]:
final_range>>arrange(_.date)

,date,pct_trips_ran
0,2022-05-01,0.515152
0,2022-05-02,0.603614
0,2022-05-03,0.907020
0,2022-05-04,0.916769
0,2022-05-05,0.893735
0,2022-05-06,0.891736
0,2022-05-07,0.479117
0,2022-05-08,0.477273
0,2022-05-09,0.598870
0,2022-05-10,0.915946


In [48]:
final_range.pct_trips_ran.mean()

0.7433618279678926

In [49]:
final_range.pct_trips_ran.median()

0.8937346437346437

In [52]:
import altair as alt

## Adding Columns

In [91]:
def get_pct_ran_df(date_list, itp_id):
    pcts = []
    for date in date_list: 
        sched_rt_df = (utils.find_ran_trips(itp_id, date))
        pct_ran = (utils.get_pct_ran(sched_rt_df, date))
        pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
        pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
        pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
        pcts.append(pct_ran)                                                    
        #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
    pcts = pd.concat(pcts)
    pct = pd.DataFrame(pcts)
    return pct


In [92]:
date_list =['2022-05-01','2022-05-02', '2022-05-31']

In [93]:
get_pct_ran_df(date_list, itp_id)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/l

,date,pct_trips_ran,n_have_rt,n_missing_rt,unmatched_rt
0,2022-05-01,0.515152,527,252,244
0,2022-05-02,0.603614,1069,252,450
0,2022-05-31,0.602144,1067,256,449


In [95]:
test = (utils.find_ran_trips(itp_id, '2022-05-01'))

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [101]:
test>>filter(_.have_rt!='both')

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,str_len,have_rt
2,4.100255e+18,6.579920e+18,881269,3481,300.0,0.0,2,2022-05-01,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,36.0,36.0,64200.0,67500.0,0.916667,NaN,NaN,NaN,NaN,NaN,left_only
3,4.100255e+18,3.465956e+17,881257,3481,300.0,0.0,2,2022-05-01,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,36.0,36.0,75900.0,78660.0,0.766667,NaN,NaN,NaN,NaN,NaN,left_only
4,4.100255e+18,-2.565743e+18,882034,3486,300.0,0.0,2,2022-05-01,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,44.0,44.0,69480.0,72300.0,0.783333,NaN,NaN,NaN,NaN,NaN,left_only
13,4.100255e+18,8.227202e+18,881372,3481,300.0,0.0,2,2022-05-01,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,39.0,39.0,29760.0,32760.0,0.833333,NaN,NaN,NaN,NaN,NaN,left_only
26,4.100255e+18,3.440335e+18,881322,3481,300.0,0.0,2,2022-05-01,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,39.0,39.0,75600.0,78900.0,0.916667,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,NaN,NaN,881136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-01,5310,6.0,right_only
1019,NaN,NaN,881201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-01,5315,6.0,right_only
1020,NaN,NaN,881130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-01,5315,6.0,right_only
1021,NaN,NaN,881211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,0.0,2022-05-01,5315,6.0,right_only


In [109]:
#reminder: left is scheduled and right is rt

In [107]:
test.have_rt.value_counts()

both          527
right_only    252
left_only     244
Name: have_rt, dtype: int64

In [102]:
test>>filter(_.have_rt!='both')>>count(_.trip_id)>>filter(_.n>1)

,trip_id,n


In [105]:
test>>filter(_.have_rt!='both')>>count(_.vehicle_id)>>filter(_.n>1)>>arrange(-_.n)

,vehicle_id,n
15,1351,11
49,2117,10
3,1313,8
26,1705,8
4,1315,7
11,1339,7
19,1503,7
20,1505,7
30,1710,7
32,1722,7


In [113]:
unmatched_rt = test>>filter(_.have_rt=='right_only')

In [118]:
## seeing if there are any matches to updates

In [111]:
updates = query_sql(f'''
SELECT * FROM `cal-itp-data-infra-staging.natalie_staging.stg_rt__trip_updates`
WHERE date = '2022-05-01' and calitp_itp_id = {itp_id}
LIMIT 1000
''')

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [115]:
join = (pd.merge(unmatched_rt, updates, how='outer', on='trip_id', indicator='_updates'))

In [117]:
join._updates.value_counts()

right_only    1000
left_only      252
both             0
Name: _updates, dtype: int64

In [119]:
#no matches

In [121]:
unmatched_rt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 771 to 1022
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   feed_key                 0 non-null      float64 
 1   trip_key                 0 non-null      float64 
 2   trip_id                  252 non-null    object  
 3   route_id                 0 non-null      object  
 4   calitp_itp_id_x          0 non-null      float64 
 5   calitp_url_number_x      0 non-null      float64 
 6   service_id               0 non-null      object  
 7   service_date             0 non-null      object  
 8   service_indicator        0 non-null      object  
 9   service_start_date       0 non-null      object  
 10  service_end_date         0 non-null      object  
 11  service_inclusion        0 non-null      object  
 12  service_exclusion        0 non-null      object  
 13  is_in_service            0 non-null      object  
 14  calitp_

In [123]:
(unmatched_rt>>select(_.trip_id,
                      _.calitp_itp_id_y,
                      _.calitp_url_number_y,
                      _.date,
                      _.vehicle_id,
                      _.str_len,
                     _.have_rt))

,trip_id,calitp_itp_id_y,calitp_url_number_y,date,vehicle_id,str_len,have_rt
771,882172,300.0,0.0,2022-05-01,1301,6.0,right_only
772,882195,300.0,0.0,2022-05-01,1301,6.0,right_only
773,882192,300.0,0.0,2022-05-01,1301,6.0,right_only
774,882193,300.0,0.0,2022-05-01,1301,6.0,right_only
775,882174,300.0,0.0,2022-05-01,1301,6.0,right_only
...,...,...,...,...,...,...,...
1018,881136,300.0,0.0,2022-05-01,5310,6.0,right_only
1019,881201,300.0,0.0,2022-05-01,5315,6.0,right_only
1020,881130,300.0,0.0,2022-05-01,5315,6.0,right_only
1021,881211,300.0,0.0,2022-05-01,5315,6.0,right_only
